# Une régression logistique sur la couleur d'un vin

L'objectif de cet exercice est de prédire la couleur d'un vin à partir de ses composants et visualiser la performance avec une courbe ROC.

## Importer les librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importer les données

Vous pouvez allez chercher les données [là](https://archive.ics.uci.edu/ml/datasets/wine+quality).  
Vous verrez que l'on a 2 tables : une pour les vins rouges et une pour les vins blancs. La première étape consistera donc à fusionner ces deux datasets pour en avoir un seul

In [ ]:
reds = pd.read_csv('winequality-red.csv',sep=";")
reds["color"]='red'
reds

In [ ]:
whites = pd.read_csv('winequality-white.csv', sep=";")
whites["color"] = 'white'
whites

In [ ]:
wines = pd.concat([reds,whites],axis=0)
wines.reset_index(drop=True, inplace=True)
wines

In [ ]:
wines.info()

In [ ]:
wines.describe()

## Un peu de dataviz

Quelques graphiques pour visualiser les distributions des différentes variables indépendantes selon la couleur du vin.

In [ ]:
fig = plt.figure(figsize=(16,18))
ax = []
for i in range(10):
    ax.append(fig.add_subplot(4,3,i+1))
    sns.boxplot(x='color',y=wines.columns[i],data=wines,palette='winter',ax=ax[i])

## Création d'un train set et test set

In [ ]:
X = wines.drop(['quality', 'color'], axis=1)
y = wines['color']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Création et entraînement du modèle

In [ ]:
from sklearn.linear_model import LogisticRegression
reglog = LogisticRegression()
reglog.fit(X_train, y_train)

## Évaluation du modèle

La première façon de vérifier que le modèle a marché consiste à regarder la matrice de confusion.

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, reglog.predict(X_test))
cm

In [ ]:
# on peut faire un affichage plus "joli" avec un DataFrame
cm = pd.DataFrame(cm, columns=['prédit ' + _ for _ in reglog.classes_])
cm.index = ['vrai ' + _ for _ in reglog.classes_]
cm

Un classifieur construit une frontière entre deux classes, la distance d'un point à la frontière constitue une information importante. Plus elle est grande, plus le modèle est confiant. Cette distance est souvent appelée *score*.

In [ ]:
score = reglog.decision_function(X_test)
score

Mais on préfère les probabilités quand elles sont disponibles :

In [ ]:
probas = reglog.predict_proba(X_test)
probas

Voyons comment le score est distribué :

In [ ]:
sc = pd.DataFrame(score, columns=['score'])
sc['color'] = y_test.values
sc.head()

In [ ]:
ax = sc['score'].hist(bins=50, figsize=(8,4))
ax.set_title('Distribution des scores de classification couleur');

On voit deux modes, probablement les deux classes. Pour en être sûr :

In [ ]:
sns.set_style('darkgrid')
ax = sc[sc.color== 'white']['score'].hist(bins=25, figsize=(8,4), label='white', color='beige', alpha=0.5)
sc[sc.color == 'red']['score'].hist(bins=25, ax=ax, label='red', color = 'mediumvioletred', alpha=0.5)
ax.set_title("Distribution des scores pour les deux classes")
ax.plot([1, 1], [0, 100], color='green', ls='--', label="frontière ?")
ax.legend();

Il y a quelques confusions autour de 0 mais le modèle est pertinent au sens où la frontière entre les deux classes est assez nette : les deux cloches ne se superposent pas. Voyons avec les probabilités :

In [ ]:
proba_1 = reglog.predict_proba(X_test)[:, 1]
pr = pd.DataFrame(proba_1, columns=['proba'])
pr['color'] = y_test.values

fig, ax = plt.subplots(1, 2, figsize=(15,5))

pr[pr.color == 'white']['proba'].hist(bins=25, label='white', color = 'beige', alpha=0.5, ax=ax[0])
pr[pr.color == 'red']['proba'].hist(bins=25, label='red', color = 'mediumvioletred', alpha=0.5, ax=ax[0])
ax[0].set_title('Distribution des probabilités des deux classes')
ax[0].plot([0.5, 0.5], [0, 1000], 'g--', label="frontière ?")
ax[0].legend();

#l'échelle logarithmique permet de mieux voir les probabilités qui sont faibles
pr[pr.color == 'white']['proba'].hist(bins=25, label='white', color = 'beige', alpha=0.5, ax=ax[1])
pr[pr.color == 'red']['proba'].hist(bins=25, label='red', color = 'mediumvioletred', alpha=0.5, ax=ax[1])
ax[1].plot([0.5, 0.5], [0, 1000], 'g--', label="frontière ?")
ax[1].set_yscale('log')
ax[1].set_title('Distribution des probabilités des deux classes\néchelle logarithmique')
ax[1].legend();

Plus l'aire commune aux deux distributions est petite, plus le modèle est confiant. Cette aire commune est reliée à la courbe [ROC](https://fr.wikipedia.org/wiki/Courbe_ROC).

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
proba = reglog.predict_proba(X_test)
fpr0, tpr0, thresholds0 = roc_curve(y_test, proba[:, 0], pos_label=reglog.classes_[0], drop_intermediate=False)

*fpr* désigne le False Positive Rate autrement dit le taux de false positive. Si la tâche est de déterminer si un vin est blanc, le taux désigne la proportion de vins rouges classés parmi les vins blancs. C'est l'erreur de classification.

*tpr* désigne le True Positive Rate c'est-à-dire le taux de True Positive.

J'ai jamais été complètement au clair sur ce que représente chacune de ces informations mais l'avantage c'est qu'on trouve toujours toutes les infos dont on a besoin le moment venu. Par exemple, [ici](https://en.wikipedia.org/wiki/Precision_and_recall)

In [ ]:
tp = pd.DataFrame(dict(fpr=fpr0, tpr=tpr0, threshold=thresholds0)).copy()
tp.drop(0, axis=0, inplace=True) #suppression du 1er seuil fixé arbitrairement à 2
tp.head(3)

In [ ]:
ax = tp.plot(x="threshold", y=['fpr', 'tpr'], figsize=(6,6))
ax.set_title("Evolution de FPR, TPR\nen fonction du seuil au delà duquel\n" + 
             "la réponse du classifieur est validée");

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
ax.plot([0, 1], [0, 1], 'k--')
# aucf = roc_auc_score(y_test == clr.classes_[0], probas[:, 0]) # première méthode
aucf = auc(fpr0, tpr0)  # seconde méthode
ax.plot(fpr0, tpr0, label='auc=%1.5f' % aucf)
ax.set_title('Courbe ROC - classifieur couleur des vins')
ax.text(0.5, 0.3, "plus mauvais que\nle hasard dans\ncette zone")
ax.legend();

La mesure [AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve) ou Area Under the Curve est l'aire sous la courbe.

Deux autres métriques sont très utilisées, la [précision](https://en.wikipedia.org/wiki/Precision_and_recall) et le [rappel](https://en.wikipedia.org/wiki/Precision_and_recall). Pour chaque classifieur, on peut déterminer un seuil *s* au delà duquel la réponse est validée avec une bonne confiance. Parmi toutes les réponses validées, la précision est le nombre de réponses correctes rapporté au nombre de réponses validées, le rappel est le nombre de réponses correctes rapportées à toutes qui aurait dû être validées. On calcule aussi la métrique *F1* qui est une sorte de moyenne entre les deux.

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, probas[:, 0], pos_label=reglog.classes_[0])

In [ ]:
pr = pd.DataFrame(dict(precision=precision, recall=recall, 
                             threshold=[0] + list(thresholds)))
pr['F1']= 2 * (pr.precision * pr.recall) / (pr.precision + pr.recall)
pr.head(n=2)

In [ ]:
ax = pr.plot(x="threshold", y=['precision', 'recall', 'F1'], figsize=(6,6))
ax.set_title("Evolution de la précision et du rappel\nen fonction du seuil au delà duquel\n" + 
             "la réponse du classifieur est validée");